In [2]:
import pandas as pd
from itertools import combinations

# Creamos un DataFrame de ejemplo
data = pd.read_csv("../../../data/aapl_1d_train.csv")
data_rsi = pd.read_excel("../../../data/1d/RSI_BOS.xlsx")
data_roc = pd.read_excel("../../../data/1d/ROC_BOS.xlsx")
data_tsi = pd.read_excel("../../../data/1d/TSI_BOS.xlsx")
data_SO = pd.read_excel("../../../data/1d/SO_BOS.xlsx")

df = pd.DataFrame()
df["RSI"] = data_rsi['Senal_RSI']
df["ROC"] = data_roc['Senal_ROC']
df["TSI"] = data_tsi['Senal_TSI']
df["SO"] = data_SO['Senal_SO']

In [3]:
# Definimos las combinaciones posibles de indicadores
indicadores = ["RSI", "ROC", "TSI", "SO"]
combinaciones = []
for r in range(1, len(indicadores) + 1):
    combinaciones.extend(combinations(indicadores, r))

# Creamos las nuevas columnas
for comb in combinaciones:
    new_column_name = '_'.join(comb)
    df[new_column_name] = df[list(comb)].apply(lambda x: ' ' if ' ' in x.values or len(set(x.values)) > 1 else x.values[0], axis=1)

df["close"] = data["Close"]
df = df.fillna(" ")

In [4]:
df

,RSI,ROC,TSI,SO,RSI_ROC,RSI_TSI,RSI_SO,ROC_TSI,ROC_SO,TSI_SO,RSI_ROC_TSI,RSI_ROC_SO,RSI_TSI_SO,ROC_TSI_SO,RSI_ROC_TSI_SO,close
0,Venta,Compra,,,,,,,,,,,,,,111.9328
1,Venta,Venta,,,Venta,,,,,,,,,,,102.5024
2,Venta,Venta,,,Venta,,,,,,,,,,,104.0032
3,Venta,Venta,,,Venta,,,,,,,,,,,94.9984
4,Venta,Venta,,,Venta,,,,,,,,,,,99.5008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5266,Venta,Venta,,Venta,Venta,,Venta,,Venta,,,Venta,,,,123.7500
5267,Venta,Venta,,Venta,Venta,,Venta,,Venta,,,Venta,,,,124.3800
5268,Venta,Venta,,,Venta,,,,,,,,,,,121.7800
5269,Venta,Venta,,,Venta,,,,,,,,,,,123.2400


In [5]:
import pandas as pd

# Supongamos que tienes un DataFrame llamado 'df' con tus 16 columnas de señales de compra y venta,
# y que también tienes una columna 'close' que contiene los precios de cierre de las acciones.

# Inicializa variables para mantener el máximo beneficio y la mejor columna
max_profit = float('-inf')  # Inicializa el máximo beneficio con un valor negativo infinito
best_column = None

# Itera sobre cada columna de señales de compra y venta
for column in df.columns:
    if column == 'close':  # Ignora la columna 'close'
        continue
    
    capital = 1000000  # Capital inicial
    position = 0  # Inicialmente, no estamos en una posición
    
    # Itera sobre cada fila del DataFrame
    for index, row in df.iterrows():
        signal = row[column]  # Obtiene la señal de compra/venta para la columna actual
        close_price = row['close']  # Precio de cierre
        
        # Lógica de compra y venta
        if signal == 'comprar' and position <= 0:
            position = 1
            entry_price = close_price
        elif signal == 'vender' and position >= 0:
            if position == 1:
                capital *= (close_price / entry_price)  # Calcula el nuevo capital después de vender
            position = 0
    
    # Si quedamos con una posición larga abierta al final del periodo, la cerramos
    if position == 1:
        capital *= (df['close'].iloc[-1] / entry_price)
    
    # Calcula el beneficio para esta columna
    profit = capital - 10000  # Restamos el capital inicial
    
    # Compara si el beneficio actual es mayor que el máximo beneficio registrado
    if profit > max_profit:
        max_profit = profit
        best_column = column

print(f"El mejor indicador en términos de beneficio es '{best_column}'.")

El mejor indicador en términos de beneficio es 'RSI'.
